In [84]:
import wrangle as w 
import pandas as pd
import numpy as np
import scipy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import regex as re
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import export_text
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from matplotlib.ticker import ScalarFormatter
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler

First I need to modify the dataframe and make a separate version of it for modeling; split, scale, fit_transform, make dataframes have the same columns (actor dummies may complicate this)

In [72]:
df = w.wrangle_df()

Using cached CSV


In [73]:
dummy_group = ['cast_actor_1', 'cast_actor_2', 'cast_actor_3', 'production_company']
dummy_df = pd.get_dummies(df.loc[:,dummy_group], drop_first = True)

In [58]:
df.columns

Index(['title', 'success', 'success_rating', 'genres', 'cast_actor_1',
       'cast_actor_2', 'cast_actor_3', 'total_n_cast', 'budget', 'revenue',
       'profit_amount', 'vote_average', 'vote_count', 'production_companies',
       'production_countries', 'overview', 'popularity', 'runtime',
       'profitable', 'release_date', 'release_year', 'imdb_id',
       'production_company', 'is_genre_adventure', 'is_genre_horror',
       'is_genre_drama', 'is_genre_scifi', 'is_genre_romance',
       'is_genre_thriller', 'is_genre_crime', 'is_genre_comedy',
       'is_genre_animation', 'is_genre_action', 'is_genre_mystery',
       'is_genre_fantasy', 'is_genre_documentary'],
      dtype='object')

In [74]:
keep =  ['budget','runtime', 'vote_average','vote_count', 'success_rating', 'success', 
         'release_year', 'is_genre_adventure', 'is_genre_horror', 'revenue',
         'is_genre_drama', 'is_genre_scifi', 'is_genre_romance',
         'is_genre_thriller', 'is_genre_crime', 'is_genre_comedy',
         'is_genre_animation', 'is_genre_action', 'is_genre_mystery',
         'is_genre_fantasy', 'is_genre_documentary', 'total_n_cast']
# success will be the only one of these that is not used as a predictor, given that it is our target
# I'm not sure if revenue would be appropriate to include. 

In [75]:
modeling_df = df.loc[:,keep]

In [76]:
# Concatenate dummy dataframe to original 
modeling_df = pd.concat([modeling_df, dummy_df], axis=1)

In [77]:
modeling_df.shape

(6893, 13018)

The data types are as they should be, as far as I know. 

In [78]:
train, validate, test = w.train_validate_test_split(modeling_df)

In [79]:
print('train:', train.shape)
print('validate:', validate.shape)
print('test:', test.shape)

train: (3859, 13018)
validate: (1655, 13018)
test: (1379, 13018)


***
Time to scale

In [141]:
X_train = train.drop(columns=['success'])
y_train = train['success']

X_validate = validate.drop(columns=['success'])
y_validate = validate['success']

X_test = test.drop(columns=['success'])
y_test = test['success']

In [131]:
# Now I need to scale. 
needs_scaling = ['budget','runtime', 'vote_average','vote_count',
                 'success_rating', 'release_year', 'revenue', 'total_n_cast']

In [142]:
#scaler = MinMaxScaler()
## Note that we only call .fit with the training data,
## but we use .transform to apply the scaling to all the data splits.
#scaler.fit(X_train[needs_scaling])
#
#X_train_scaled = pd.DataFrame(scaler.transform(X_train[needs_scaling]))
#X_validate_scaled = pd.DataFrame(scaler.transform(X_validate[needs_scaling]))
#X_test_scaled = pd.DataFrame(scaler.transform(X_test[needs_scaling]))

scaler = MinMaxScaler()
# Note that we only call .fit with the training data,
# but we use .transform to apply the scaling to all the data splits.
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_validate_scaled = scaler.transform(X_validate)
X_test_scaled = scaler.transform(X_test)

X_train_s = pd.DataFrame(X_train_scaled, columns = X_train.columns)
X_validate_s = pd.DataFrame(X_validate_scaled, columns = X_validate.columns)
X_test_s = pd.DataFrame(X_test_scaled, columns = X_train.columns)

In [143]:
print('nulls in train scaled', X_train_scaled.isna().sum().sum())
print('nulls in validate scaled', X_validate_scaled.isna().sum().sum())
print('nulls in test scaled', X_test_scaled.isna().sum().sum())

nulls in train scaled 0
nulls in validate scaled 0
nulls in test scaled 0


In [144]:
X_train_scaled

,0,1,2,3,4,5,6,7,8,9,...,13007,13008,13009,13010,13011,13012,13013,13014,13015,13016
0,0.046759,0.328125,0.747253,0.069982,0.167294,0.676471,0.0,0.0,0.113474,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.023346,0.343750,0.769231,0.035524,0.061627,0.862745,0.0,0.0,0.021657,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.033380,0.434375,0.780220,0.002416,0.024742,0.764706,0.0,0.0,0.011787,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.190581,0.371875,0.670330,0.120142,0.005398,0.960784,0.0,0.0,0.015782,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.086895,0.309375,0.604396,0.007034,0.003089,0.872549,0.0,0.0,0.004663,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3854,0.006496,0.337500,0.725275,0.006892,0.007827,0.794118,0.0,0.0,0.001080,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3855,0.053448,0.325000,0.703297,0.087673,0.059997,0.970588,0.0,0.0,0.049004,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3856,0.030035,0.315625,0.703297,0.001847,0.000153,0.911765,0.0,0.0,0.000073,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3857,0.030035,0.353125,0.846154,0.003197,0.000686,0.774510,0.0,0.0,0.000274,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [134]:
#dropping original values merging scaled values for numerical columns
X_train = X_train.drop(columns = needs_scaling, axis = 1)
X_validate = X_validate.drop(columns = needs_scaling, axis = 1)
X_test = X_test.drop(columns = needs_scaling, axis = 1)

In [135]:
X_train = X_train.merge(X_train_scaled, left_index=True, right_index=True, how = "left")
X_validate = X_validate.merge(X_validate_scaled, left_index=True, right_index=True, how = "left")
X_test = X_test.merge(X_test_scaled, left_index=True, right_index=True, how = "left")
#X_train = pd.concat([X_train, X_train_scaled], axis=0)

In [31]:
train['baseline_prediction'] = 0
validate['baseline_prediction'] = 0
test['baseline_prediction'] = 0

/var/folders/qn/sd1_3x2j4196db03067vql600000gn/T/ipykernel_13666/3381084422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['baseline_prediction'] = 0


In [42]:
predictors = ['budget','runtime',
              'vote_average','vote_count','release_year',
              'is_genre_adventure', 'is_genre_horror',
              'is_genre_drama', 'is_genre_scifi', 'is_genre_romance',
              'is_genre_thriller', 'is_genre_crime', 'is_genre_comedy',
              'is_genre_animation', 'is_genre_action', 'is_genre_mystery',
              'is_genre_fantasy', 'is_genre_documentary', 'total_n_cast']


X_train = train[predictors].values
y_train = train['success'].values

X_validate = validate[predictors].values
y_validate = validate['success'].values

X_test = test[predictors].values
y_test = test['success'].values


In [46]:
X_dummy.columns

Index(['budget', 'runtime', 'vote_average', 'vote_count', 'release_year',
       'is_genre_adventure', 'is_genre_horror', 'is_genre_drama',
       'is_genre_scifi', 'is_genre_romance',
       ...
       'cast_actor_3_Zoey Deutch', 'cast_actor_3_Zooey Deschanel',
       'cast_actor_3_Zoé Félix', 'cast_actor_3_Zoë Kravitz',
       'cast_actor_3_Zulay Henao', 'cast_actor_3_Àlex Brendemühl',
       'cast_actor_3_Émilie Dequenne', 'cast_actor_3_Мария Бурова',
       'cast_actor_3_Моррис Честнат', 'cast_actor_3_张雨绮'],
      dtype='object', length=8544)

In [45]:
# print grid search results
def print_cv_results(gs, title):

    print(title)

    print(f'Best Score = {gs.best_score_:.4f}')
    print(f'Best Hyper-parameters = {gs.best_params_}')
    print()

    print('Test Scores:')
    test_means = gs.cv_results_['mean_test_score']
    test_stds = gs.cv_results_['std_test_score']
    for mean, std, params in zip(test_means, test_stds, gs.cv_results_['params']):
        print(f'{mean:.4f} (+/-{std:.3f}) for {params}')
    print()

    print('Training Scores:')
    train_means = gs.cv_results_['mean_train_score']
    train_stds = gs.cv_results_['std_train_score']
    for mean, std, params in zip(train_means, train_stds, gs.cv_results_['params']):
        print(f'{mean:.4f} (+/-{std:.3f}) for {params}')
        
        # save grid search results to file
def save_cv_results(gs, title, fileName):
    with open(fileName, 'a') as f:

        print(title, file=f)

        print(f'Best Score = {gs.best_score_:.4f}', file=f)
        print(f'Best Hyper-parameters = {gs.best_params_}', file=f)
        print('', file=f)

        print('Test Scores:', file=f)
        test_means = gs.cv_results_['mean_test_score']
        test_stds = gs.cv_results_['std_test_score']
        for mean, std, params in zip(test_means, test_stds, gs.cv_results_['params']):
            print(f'{mean:.4f} (+/-{std:.3f}) for {params}', file=f)
        print('', file=f)

        print('Training Scores:', file=f)
        train_means = gs.cv_results_['mean_train_score']
        train_stds = gs.cv_results_['std_train_score']
        for mean, std, params in zip(train_means, train_stds, gs.cv_results_['params']):
            print(f'{mean:.4f} (+/-{std:.3f}) for {params}', file=f)

In [48]:
# Logistic Regression
logReg = LogisticRegression()
c_list = [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
param_grid = {'C': c_list,
              'solver': ['newton-cg', 'lbfgs', 'sag', 'saga']}

gs = GridSearchCV(estimator=logReg,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=10,
                  return_train_score=True)
#gs = gs.fit(X_dummy_train, y_train)
gs = gs.fit(X_train, y_train)
print_cv_results(gs, 'Logistic Regression Accuracy')

/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1344, in fit
    X, y = self._validate_data(X, y, accept_sparse='csr', dtype=_dtype,
  File "/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/base.py", line 433, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py", line 871, in 

ValueError: could not convert string to float: 'ParamountPictures'